In [1]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf

import warnings
warnings.filterwarnings('ignore')

In [2]:
datapath = '/Users/yufanwang/Desktop/MarmosetWM_Project/revision_NC/af_projection/result'

In [ ]:
human_af_l_all = np.loadtxt(f'{datapath}/probtrackx2/human_af_l_thr0_norm.txt')
chimp_af_l_all = np.loadtxt(f'{datapath}/probtrackx2/chimp_af_l_thr0_norm.txt')
macaque_af_l_all = np.loadtxt(f'{datapath}/probtrackx2/macaque_tvb_af_l_thr0_norm.txt')
marmoset_af_l_all = np.loadtxt(f'{datapath}/probtrackx2/marmoset_MBM_af_l_thr0_norm.txt')
marmoset_Paxinos_af_l_all = np.loadtxt(f'{datapath}/Paxinos/marmoset_MBM_af_l_thr0_norm_Paxinos.txt')

human_af_r_all = np.loadtxt(f'{datapath}/probtrackx2/human_af_r_thr0_norm.txt')
chimp_af_r_all = np.loadtxt(f'{datapath}/probtrackx2/chimp_af_r_thr0_norm.txt')
macaque_af_r_all = np.loadtxt(f'{datapath}/probtrackx2/macaque_tvb_af_r_thr0_norm.txt')
marmoset_af_r_all = np.loadtxt(f'{datapath}/probtrackx2/marmoset_MBM_af_r_thr0_norm.txt')
marmoset_Paxinos_af_r_all = np.loadtxt(f'{datapath}/Paxinos/marmoset_MBM_af_r_thr0_norm_Paxinos.txt')

In [ ]:
index_human_chimp_L = np.loadtxt(f'/Users/yufanwang/Desktop/MarmosetWM_Project/revision_NC/indi_bp_atlas/index_human_chimp_L.txt')
index_human_macaque_L = np.loadtxt(f'/Users/yufanwang/Desktop/MarmosetWM_Project/revision_NC/indi_bp_atlas/index_human_macaque_L.txt')
index_human_marmoset_L = np.loadtxt(f'/Users/yufanwang/Desktop/MarmosetWM_Project/revision_NC/indi_bp_atlas/index_human_marmoset_L.txt')

## construct data

In [8]:
def read_subject_list(txt_path):
    with open(txt_path, "r") as f:
        subs = [line.strip() for line in f if line.strip()]
    return subs

def build_species_df_atlas(list_path, species_name, atlas, left_vals, right_vals, region):
    subj = read_subject_list(list_path)
    L = np.asarray(left_vals).reshape(-1)
    R = np.asarray(right_vals).reshape(-1)

    if not (len(subj) == len(L) == len(R)):
        raise ValueError(
            f"[{species_name}] length mismatch: "
            f"subjects={len(subj)}, L={len(L)}, R={len(R)}"
        )

    df_L = pd.DataFrame({
        "subject_id": subj,
        "species": species_name,
        "atlas": atlas,
        "hemisphere": "LH",
        "region": region,
        "connectivity_score": L
    })

    df_R = pd.DataFrame({
        "subject_id": subj,
        "species": species_name,
        "atlas": atlas,
        "hemisphere": "RH",
        "region": region,
        "connectivity_score": R
    })

    return pd.concat([df_L, df_R], ignore_index=True)

In [10]:
human_list_path = "/Users/yufanwang/Desktop/MarmosetWM_Project/revision_NC/indi_bp_atlas/humanlist40.txt"
chimp_list_path = "/Users/yufanwang/Desktop/MarmosetWM_Project/revision_NC/indi_bp_atlas/chimplist46.txt"
macaque_list_path = "/Users/yufanwang/Desktop/MarmosetWM_Project/revision_NC/indi_bp_atlas/macaque_tvb_list.txt"
marmoset_list_path = "/Users/yufanwang/Desktop/MarmosetWM_Project/revision_NC/indi_bp_atlas/marmoset_MBM_list.txt"

i = 17
region = "A45c"

roi_index_human = i - 1
roi_index_chimp = index_human_chimp_L[roi_index_human] - 1
roi_index_macaque = index_human_macaque_L[roi_index_human] - 1
roi_index_marmoset = index_human_marmoset_L[roi_index_human] - 1
roi_index_marmoset_Paxinos = 31 - 1

value_human_L = human_af_l_all[:,roi_index_human]
value_chimp_L = chimp_af_l_all[:,int(roi_index_chimp)]
value_macaque_L = macaque_af_l_all[:,int(roi_index_macaque)]
value_marmoset_L = marmoset_af_l_all[:,int(roi_index_marmoset)]
value_marmoset_Paxinos_L = marmoset_Paxinos_af_l_all[:,int(roi_index_marmoset_Paxinos)]

value_human_R = human_af_r_all[:,roi_index_human]
value_chimp_R = chimp_af_r_all[:,int(roi_index_chimp)]
value_macaque_R = macaque_af_r_all[:,int(roi_index_macaque)]
value_marmoset_R = marmoset_af_r_all[:,int(roi_index_marmoset)]
value_marmoset_Paxinos_R = marmoset_Paxinos_af_r_all[:,int(roi_index_marmoset_Paxinos)]

df = pd.concat([
    build_species_df_atlas(human_list_path,    "human",      "BNA", value_human_L,    value_human_R, region),
    build_species_df_atlas(chimp_list_path,    "chimpanzee", "BNA", value_chimp_L,    value_chimp_R, region),
    build_species_df_atlas(macaque_list_path,  "macaque",    "BNA", value_macaque_L,  value_macaque_R, region),
    build_species_df_atlas(marmoset_list_path, "marmoset",   "BNA", value_marmoset_Paxinos_L, value_marmoset_Paxinos_R, region),
], ignore_index=True)

print(df.head())
print(df.shape)  # 应该是 2 * (40+46+8+24) = 236 行

df.to_csv(f'{datapath}/af_projection_{region}_Paxinos.csv', index=False)

df_marmoset = pd.concat([
    build_species_df_atlas(marmoset_list_path, "marmoset", "BNA", value_marmoset_L, value_marmoset_R, region),
    build_species_df_atlas(marmoset_list_path, "marmoset", "Paxinos", value_marmoset_Paxinos_L, value_marmoset_Paxinos_R, region),
], ignore_index=True)

print(df_marmoset.head())
print(df_marmoset.shape)  # 应该是 2 * (40+46+8+24) = 236 行

df_marmoset.to_csv(f'{datapath}/af_projection_{region}_Paxinos_marmoset.csv', index=False)

  subject_id species atlas hemisphere region  connectivity_score
0     100307   human   BNA         LH   A45c            0.335231
1     100408   human   BNA         LH   A45c            0.339382
2     103414   human   BNA         LH   A45c            0.133474
3     105115   human   BNA         LH   A45c            0.264675
4     106016   human   BNA         LH   A45c            0.541276
(236, 6)
   subject_id   species atlas hemisphere region  connectivity_score
0  sub-NIHm14  marmoset   BNA         LH   A45c            0.180736
1  sub-NIHm15  marmoset   BNA         LH   A45c            0.050014
2  sub-NIHm16  marmoset   BNA         LH   A45c            0.149554
3  sub-NIHm17  marmoset   BNA         LH   A45c            0.158284
4  sub-NIHm19  marmoset   BNA         LH   A45c            0.126013
(96, 6)
